In [ ]:
import scipy
matlab = scipy.io.loadmat('Xtrain.mat')

In [ ]:
X_train = matlab['Xtrain']

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Convert the data to a list
X_train_list = X_train.flatten().tolist()
print(X_train_list)

# Plot a line of the values in the training set
fig = px.line(y=X_train_list, x=range(len(X_train_list)), title='Line Plot of X_train Values')
fig.update_layout(xaxis_title='Index', yaxis_title='Value') 
fig.show()

In [ ]:
import pandas as pd

In [ ]:
def create_features(timeseries_array, n_lags):
    df = pd.DataFrame(timeseries_array, columns=['label'])
    for i in range(1, n_lags + 1):
        df[f'lag {i}'] = df['label'].shift(i)
    df = df.dropna()

    X = df.drop(columns=['label']).to_numpy()
    y = df['label'].to_numpy()

    return X, y

X, y = create_features(X_train_list, 10)

print(X.shape)
print(y.shape)

In [ ]:
def expanding_window_cv_sets(X, y, folds: int, validation_split_ratio: float = 0.2):
    
    n = len(X)
    fold_size = int(n / folds)

    for fold in range(folds):
        start_fold = fold * fold_size
        end_fold = start_fold + fold_size

        training_start = 0
        training_end = int(end_fold - fold_size * validation_split_ratio)

        validation_start = training_end

        training_set = X[training_start:training_end], y[training_start:training_end]
        validation_set = X[validation_start:end_fold], y[validation_start:end_fold]
        yield training_set, validation_set

# Example usage
folds = 5
for train_set, val_set in expanding_window_cv_sets(X, y, folds):
    train_X, train_y = train_set
    val_X, val_y = val_set

    print("Training Set:")
    print(train_X.shape, train_y.shape)
    print("Validation Set:")
    print(val_X.shape, val_y.shape)
    print("\n")


In [ ]:
# Create two tables to store the results of CV grid search
epoch_grid_search_results = pd.DataFrame(columns=['epoch', 'fold', 'MSE', 'MAE'])
lag_grid_search_results = pd.DataFrame(columns=['lag', 'fold', 'MSE', 'MAE'])
hidden_units_grid_search_results = pd.DataFrame(columns=['hidden_units', 'fold', 'MSE', 'MAE'])

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.reshape(-1, 1))


folds = 5
for epoch in [10, 20, 50, 100, 200, 500, 1000, 2000]:
    print("Epoch: ", epoch)
    for train_set, val_set in expanding_window_cv_sets(X_scaled, y_scaled, folds):
        train_X, train_y = train_set
        val_X, val_y = val_set
        
        # Reshape the data to be 3D for LSTM input
        train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
        val_X = val_X.reshape((val_X.shape[0], 1, val_X.shape[1]))

        # First, we are gonna fix the LSTM to 10 hidden units and grid search the epochs

        # Define the lSTM model with 10 hidden units
        model = Sequential([
            LSTM(10, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=False),
            Dense(1)
        ])

        model.compile(optimizer='adam', loss='mse')
        model.fit(train_X, train_y, epochs=epoch, batch_size=32, verbose=0)

        # Recursively predict the validation set, so start with the first example of the validation set and use the model to predict the next value, then use that value to predict the next one, and so on.
        current_input = val_X[0].reshape((1, 1, val_X.shape[2]))
        y_pred = []

        for i in range(len(val_y)):
            new_y = model.predict(current_input)
            y_pred.append(new_y[0][0])
            # Remove the first value of the current input and append the new value to the end of the input sequence
            current_input = np.append(current_input[:, :, 1:], new_y.reshape((1, 1, 1)), axis=2)
        
        y_pred = np.array(y_pred).reshape(-1, 1)
        print("Epochs: ", epoch)
        print("Standardized MSE: ", mean_squared_error(val_y, y_pred))
        print("Standardized MAE: ", np.mean(np.abs(val_y - y_pred)))
        print("\n")
        
        # Append the results to the epoch grid search results table
        epoch_grid_search_results = pd.concat([epoch_grid_search_results, pd.DataFrame({'epoch': [epoch], 'MSE': [mean_squared_error(val_y, y_pred)], 'MAE': [np.mean(np.abs(val_y - y_pred))]})], ignore_index=True)
        


   


